In [11]:
import pandas as pd
import requests

In [12]:
#----------------------------------------------------------------------------------------
#
#
# 1. Oculomotor event detection (i.e., Fixations and saccades)
# 
#
#----------------------------------------------------------------------------------------

In [13]:
# Read gaze data with AOIs using pandas library
data = pd.read_csv('data/gazeDataWithAOIs.csv')
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (8,9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
# Preview data
data.head()

,Row,Respondent,Age,Gender,Group,Calibration,Timestamp,EventSource,SlideEvent,StimType,Duration,CollectionPhase,SourceStimuliName,EventSource.1,ET_GazeLeftx,ET_GazeLefty,ET_GazeRightx,ET_GazeRighty,ET_PupilLeft,ET_PupilRight,ET_TimeSignal,ET_DistanceLeft,ET_DistanceRight,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula
0,1,Anonymous 14-11-22 09h35m,0,OTHER,Default,Excellent,122.0544,1.0,StartSlide,TestImage,900000.0,StimuliDisplay,img,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Anonymous 14-11-22 09h35m,0,OTHER,Default,Excellent,128.9448,NaN,NaN,NaN,NaN,NaN,img,1.0,951.0,486.0,970.0,503.0,NaN,NaN,141.658,566.252441,569.685181,0.598863,0.525673,0.429841,0.519144,0.0,0.0,960.5,494.5,0,0,0,1,0,0,1,0,0,0,0,0
2,3,Anonymous 14-11-22 09h35m,0,OTHER,Default,Excellent,132.9790,1.0,StartMedia,TestImage,900000.0,StimuliDisplay,img,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
3,4,Anonymous 14-11-22 09h35m,0,OTHER,Default,Excellent,137.2724,NaN,NaN,NaN,NaN,NaN,img,1.0,938.0,500.0,966.0,519.0,NaN,NaN,149.984,566.216125,569.593445,0.599070,0.525453,0.429996,0.518994,0.0,0.0,952.0,509.5,0,0,0,1,0,0,1,0,0,0,0,0
4,5,Anonymous 14-11-22 09h35m,0,OTHER,Default,Excellent,145.6203,NaN,NaN,NaN,NaN,NaN,img,1.0,938.0,500.0,NaN,NaN,2.614349,NaN,158.333,566.265381,NaN,0.599004,0.525297,NaN,NaN,0.0,4.0,938.0,500.0,0,0,0,1,0,0,1,0,0,0,0,0


In [15]:
#Keep only the gaze data coming from the eye-tracking device
data = data[ data['EventSource.1'] == 1 ]

In [16]:
#Fill n/a values with -1 (for JSON Compliance in requests) 
data = data.fillna(-1) 

In [21]:
# Oculomotor event detection (i.e., Fixations and saccades)
# Using customizable I-VT (Velocity Threshold) filter for fixation and saccade detection

dataWithEvents = None

# Iteratate over the participants and the apply I-VT filter on the data of each participant
for participant in data['Respondent'].unique():
    
    # filter in the participant data
    ParticipantData = data[data['Respondent']==participant]
    
    print(f'Applying fixation filter on the data for participant: {participant}')
    config = {
                    "fixationFilter" : "IVT",
                    "screenResolutionWidth" : 1920, 
                    "screenResolutionHeight" : 1080, 
                    "monitorSize" : 17, 
                    "screenDistance" : 70,
                    "gapFill" : True, 
                    "maxGapLength": 75, 
                    "noiseReduction" : False, 
                    "filterType" : "Median", 
                    "windowNoise" : 3,
                    "windowVelocity" : 20, 
                    "velocityThreshold" : 30, 
                    "mergeFixation" : True, 
                    "maxTimeBtwFixation" : 75,
                    "maxAngleBtwFixation" : 0.5, 
                    "discardShortFixation" : True,
                    "minDurationFixation" : 60
    }

    # POST pariticipant gaze data to the (R) eventDetection server
    # url = 'http://rserver:6789/eventDetection'
    url = 'http://127.0.0.1:6789/eventDetection'
    payload = {'config': config, 'data': ParticipantData.to_dict(orient="records")}
    response = requests.post(url, json = payload)
    
    # Create new a dataframe from the json response of the (R) eventDetection server
    participantDataWithEvents = pd.DataFrame.from_records(response.json())
    
    #Assign participantDataWithEvents to dataWithEvents or concate
    if dataWithEvents is None: 
        dataWithEvents = participantDataWithEvents
    else:
        dataWithEvents = pd.concat([dataWithEvents, participantDataWithEvents], axis=0)
        
    print('... Complete')

Applying fixation filter on the data for participant: Anonymous 14-11-22 09h35m
... Complete
Applying fixation filter on the data for participant: Anonymous 17-11-22 21h53m
... Complete


In [22]:
# Preview gaze data with event information
dataWithEvents.head()

,Row,Respondent,Age,Gender,Group,Calibration,Timestamp,SourceStimuliName,EventSource.1,ET_TimeSignal,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula,InterpolatedGazeX,InterpolatedGazeY,InterpolatedDistance,GazeVelocityAngle,FixID,Fixation Index by Stimulus,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,ET_PupilLeft,GazeAccelerationAngle,ET_PupilRight,SacID,Saccade Index by Stimulus,Saccade Start,Saccade End,Saccade Duration,Saccade Amplitude,Saccade Peak Velocity,Saccade Peak Acceleration,Saccade Peak Deceleration,Saccade Direction
0,2,Anonymous 14-11-22 09h35m,0,OTHER,Default,Excellent,128.9448,img,1,141.658,0.5989,0.5257,0.4298,0.5191,0,0,960.5,494.5,0,0,0,1,0,0,1,0,0,0,0,0,960.5,494.5,567.9688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,Anonymous 14-11-22 09h35m,0,OTHER,Default,Excellent,137.2724,img,1,149.984,0.5991,0.5255,0.4300,0.5190,0,0,952.0,509.5,0,0,0,1,0,0,1,0,0,0,0,0,952.0,509.5,567.9048,27.4655,1.0,,947.656,491.5955,133.1086,358.082,224.9734,0.3205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,Anonymous 14-11-22 09h35m,0,OTHER,Default,Excellent,145.6203,img,1,158.333,0.5990,0.5253,NaN,NaN,0,4,938.0,500.0,0,0,0,1,0,0,1,0,0,0,0,0,938.0,500.0,566.2654,13.3622,1.0,,947.656,491.5955,133.1086,358.082,224.9734,0.3205,2.6143,-1689.4409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,Anonymous 14-11-22 09h35m,0,OTHER,Default,Excellent,153.9470,img,1,166.660,0.5989,0.5252,0.4299,0.5187,0,0,948.0,499.0,0,0,0,1,0,0,1,0,0,0,0,0,948.0,499.0,567.7859,41.4435,1.0,,947.656,491.5955,133.1086,358.082,224.9734,0.3205,NaN,3372.4442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,Anonymous 14-11-22 09h35m,0,OTHER,Default,Excellent,162.2703,img,1,174.983,0.5991,0.5251,0.4301,0.5187,0,0,969.5,515.0,0,0,0,1,0,0,1,0,0,0,0,0,969.5,515.0,567.7856,36.3849,1.0,,947.656,491.5955,133.1086,358.082,224.9734,0.3205,NaN,-607.7654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# export dataWithEvents as csv
dataWithEvents.to_csv("data/gazeDataWithAOIsAndEvents.csv",  index=False)